# **Agents and Tools with LangGraph**

### Dr. Santosh Chapaneri
### Lead AI Product Engineer, Wolters Kluwer


In [1]:
!pip install langgraph==0.3.2 mistralai pandas langchain==0.2.5

  Using cached langgraph-0.3.2-py3-none-any.whl.metadata (17 kB)
Using cached langgraph-0.3.2-py3-none-any.whl (130 kB)
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling langgraph-1.0.7:
      Successfully uninstalled langgraph-1.0.7


In [2]:
import os

os.environ["MISTRAL_API_KEY"] = "T5DdItPzE5WqcQrrOL7RLlpvJHQWoGHY"
api_key = os.environ["MISTRAL_API_KEY"]

In [3]:
import json
import pandas as pd
from mistralai import Mistral
import functools
from langgraph.graph import StateGraph, END

In [5]:
# === DATA ===
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

df = pd.DataFrame(data)
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


In [6]:
# === TOOL FUNCTIONS ===
def retrieve_payment_status(df: pd.DataFrame, transaction_id: str) -> str:
    if transaction_id in df['transaction_id'].values:
        status = df.loc[df['transaction_id'] == transaction_id, 'payment_status'].values[0]
        return f"The payment status for transaction {transaction_id} is: {status}"
    else:
        return f"Transaction ID {transaction_id} not found."

def retrieve_payment_date(df: pd.DataFrame, transaction_id: str) -> str:
    if transaction_id in df['transaction_id'].values:
        date = df.loc[df['transaction_id'] == transaction_id, 'payment_date'].values[0]
        return f"The payment date for transaction {transaction_id} is: {date}"
    else:
        return f"Transaction ID {transaction_id} not found."

names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df)
}

In [7]:
# === TOOL SPECIFICATIONS ===
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Retrieve the payment status for a given transaction ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {"type": "string", "description": "The ID of the transaction"}
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Retrieve the payment date for a given transaction ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {"type": "string", "description": "The ID of the transaction"}
                },
                "required": ["transaction_id"],
            },
        },
    }
]

In [ ]:
# === MISTRAL CLIENT ===
model = None

client = None

In [ ]:
# === GRAPH STATE ===
State = None

In [ ]:
# === NODES ===

def planner(state: State) -> State:
    None

def execute_tools(state: State) -> State:
    "Executes the tools called by the model and adds the results to the state."
    None

    for tool_call in tool_calls:
        None

        if function_name in names_to_functions:
            None
        else:
            None

    None


def final_response(state: State) -> State:
    None

def check_tool_needed(state: State) -> str:
    None

In [ ]:
# === GRAPH ===
graph_builder = None


In [ ]:
# === RUN Query ===
def run_query(user_query: str, with_system: bool = True):
    messages = None
    if with_system:
        messages.append(
            {
                "role": None,
                "content": "You are a helpful assistant that provides information \
                about payment transactions."
            }
        )
    messages.append(
        {
            "role": None,
            "content": None
        }
    )

    state = None
    result = None

    return None

In [ ]:
user_query = "What's the status of my transaction T1001?"
# user_query = "When was transaction T1005 made?"
# user_query = "who is prime minister of india"

print(None)